## Detta steg förutsätter att kodboken är kontrollerad - CheckExcelfiles ska ha körts. Om det finns fel i mappningen så stannar Python där och detta steg får inte köras

1. Läser in de nya excelfilerna i Pandas. 
2. Mappar mot kodboken. 
3. Skapar nödvändiga varibler 
4. Flyttar de inlästa excelfilerna till katalogen inlästa
5. Skapar nya excelfiler från template
6. Laddar databasen
7. Sparar undan denna som ett pickle objekt


In [3]:
import shutil, os, zipfile
import pandas as pd
import numpy as np
import datetime as datetime
import os

In [4]:
cpath = os.getcwd()

__Börjar med att skapa en felhanteringsklass som används för att checka om aktuella datum redan laddats i databasen__

In [5]:
class Error(Exception):
 
    # Error is derived class for Exception, but
    # Base class for exceptions in this module
    pass

# Ett generiskt error som vi kan lägga in när vi behöver stänga Python
class ValueError(Error):
    def __init__(self, msg):
        self.msg = msg


### Läser in de nya excelfilerna i Pandas -ligger på inputkatalogen

In [6]:
# Läser in excelfilerna i Pandas format, sparar i en lista
df_list = []
for folderName, subfolders, fileNames in os.walk(os.path.join(cpath,"Input")):
    for name in fileNames:
        df = pd.read_excel(os.path.join(folderName,name), sheet_name = 0)
        if len(df[pd.isnull(df.Kanal) == True]):
            raise(ValueError('Kanal saknar ett värde. Inputfil ' + name))
        if len(df[pd.isnull(df.Nyhetskategori) == True]):
            raise(ValueError('Nyhetskategori saknar ett värde. Inputfil ' + name))
        
        df_list.append(df)

### Läser in kodboken och skapar dictionary för mappning

In [9]:
# Kanal, senaste kodboken
meta1_df = pd.read_excel(os.path.join(cpath,'kodbok', 'kodbok.xls'), sheet_name = 1)

# Nyhetskategori
meta2_df = pd.read_excel(os.path.join(cpath,'kodbok', 'kodbok.xls'), sheet_name = 4)

mapping_kanal = meta1_df.set_index('kod')['kanal'].to_dict()
mapping_nykat = meta2_df.set_index('kod')['kategori'].to_dict()

In [10]:
### Beräkbar index, gör en funktion som skapar variabeln seg. Denna används sedan för att beräkna index
### Skapar index på det nya måttet. Använder 3.56 som bas, medelvärde för segmentsvariabeln 2012 

def segment(mfyll,impact):
    if mfyll in [1,2,3] and impact in [5,6]:
        seg = 1
    elif mfyll in [1,2,3] and impact == 4:
        seg = 2;
    elif mfyll in [1,2,3] and impact in [1,2,3]:
        seg = 3
    elif mfyll in [4,5,6,7] and impact in [1,2,3]:
        seg = 4
    elif mfyll in [4] and impact in [4]:
        seg = 5
    elif mfyll in [5,6,7] and impact in [4]:
        seg = 6
    elif mfyll in [4,5,6,7] and impact in [5,6]:
        seg = 7  
    index = round((seg / 3.56) * 100) 
    return index

__Läser in loaddate och skapar dictionary för mappning. Denna håller namn på de datum som ska läsas in__

In [11]:
# Egendefinierad dataframe med laddningsdatum och filnamn

load_date_df = pd.read_pickle("load_df.pkl")


In [13]:
# Nu skapar vi ett dictionary som kan användas för mappning 
mapping_vecka = load_date_df.set_index('Date')['vecka'].to_dict()

In [23]:
len(df_list)

12

In [14]:
### Nu loopar vi igenom de inlästa pandas filerna och skapar de variabler som behöver läggas till
### De transformerade dataframse läggs i en ny lista
# Här måste vi lägga in vecka, hämta från metadata. Skapa ett dicionaru från metdata som håller datum:vecka och applicera 
# logiken nedan med en apply, lambda, map logik

transformed_list = []
i = -1
for df in df_list:
    i +=1
    if df.iloc[0,0] != 0: #Tar bort de som inte har några rapporterade nyheter
        df['nykat1'] = df['Nyhetskategori'].map(mapping_nykat)
        df['kanal1'] = df['Kanal'].map(mapping_kanal)
        df['date'] = pd.to_datetime('20'+fileNames[i][0:6])
        df['vers'] = fileNames[i][7:9]
        df['year'] = df['date'].dt.year
        df['vecka'] = df['date'].map(mapping_vecka)

        df = df.rename(columns = {'Måluppfyllelse' :'mfyll', 'Aktör': 'Akt_r', 'Rubrik':'rubrik',
                                     'Nyhetskategori':'nykat', 'Medieslag': 'med'})
        df['index'] = df.apply(lambda x : segment(x['mfyll'], x['Impact']), axis = 1)   
        transformed_list.append(df)

In [15]:
### Läser vi in existerande databasen från Pickles format

In [16]:
databas = pd.read_pickle('Databas.pkl')

In [ ]:
### Nu ska vi appenda på de 12 filerna på databasen. Sedan kan rapporterna tas fram
### Här måste vi lägga in en kontroll på att vi inte laddar in ett existerande datum
### Då ska laddningen avbrytas

In [17]:
# Skapar ett set av alla de datum som är inlästa i databesen
date_set = set()
for date in databas.date:
    date_set.add(date)

In [18]:
# Nu gör vi en lista av de datum som ligger i inputfilerna

input_date = list()
for i in range(len(transformed_list)):
    i_set = transformed_list[i].iloc[0].date
    input_date.append(i_set)

In [19]:
# Nu checkar vi varje datum med setet för redan laddade datum
# Om True faller ut ska Python stängas ned, så att databasen inte laddas felaktigt

for date in input_date:
    if date in date_set:
            raise(ValueError('Datumet har redan laddats i befintlig databas - inte tillåtet att ladda om'))

__Konkatinera filerna mot databasen - måste lägga databasen i en lista också__

In [ ]:
# Konkatinerar varje fil på databasen
# Lägger varje uppdaterad databas i en egen lista
# När den sista filen är kontatinerad är databasen fullständigt uppdaterad
db_list = []
for i in range(len(transformed_list)):
    databas = pd.concat([databas,transformed_list[i]])

In [ ]:
# Nu sparar vi undan databasen som ett Pickle format - det är denna vi läser in och uppdaterar nästa vecka

databas.to_pickle("databas.pkl")

### När databasen laddad. Flytta på de inlästa filerna och lägg upp nya, tomma filer, skapade från template filen.

In [ ]:
# Ny scannar vi igenom alla filer på old catalog och flyttat dem till den nya katalogen
for folderName, subfolders, fileNames in os.walk(os.path.join(cpath,"Input")):
    for names in fileNames:
        shutil.move(os.path.join(cpath,folderName,names), os.path.join(cpath,"InputLoaded"))

__Lägg upp nya tomma excelfiler. Plocka ut maxdatum på uppdaterad databas. Härled sedan de sju efterföljande dagarna. Det är dessa som ska läsas in från filen load_date_df som uppdateras varje år__

In [ ]:
maxdate = databas.date.max()

In [ ]:
## Skapar lista med de sju efterföljande dagarna 
date_list = []
for i in range(7):
    i +=1
    date = maxdate + datetime.timedelta(days= i)
    date_list.append(date)   

In [ ]:
## Plockar dessa datum från load_date_df, som innehåller filnam. Nyckel på date

filename_df = load_date_df[load_date_df['Date'].isin(date_list)]

In [ ]:
name_list = list(filename_df['filnamn'])


In [ ]:
# Lägger upp nya filer
for name in name_list:
    shutil.copy(os.path.join(cpath,"template","template.xls"),os.path.join(cpath,"Input",name))
    

In [ ]:
# Först tar vi bort den existerande kodboken

os.unlink(os.path.join(cpath,"kodbok","existing_kodbok.xls"))

In [ ]:
# Sedan döper vi om kodbok till existing_kodbok.xls - nu är det den som är master

shutil.move(os.path.join(cpath,"kodbok","kodbok.xls"),os.path.join(cpath,"kodbok","existing_kodbok.xls"))